<a href="https://colab.research.google.com/github/RomanaNourin/Crime-Activity-Detection/blob/main/HBD_CNN%2BAlexNet_Adam(0_001).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.15


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pafy

In [ ]:
!pip install youtube-dl # install the missing module


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.1 MB/s eta 0:00:00


In [ ]:
import os
import cv2

# Base directory containing the dataset (Train and Test folders)
base_dir = '/content/drive/MyDrive/Human Behavior dataset (HBD21)/Enhanced_test&train_data'

# Directory where frames will be saved
output_dir = '/content/drive/MyDrive/Human Behavior dataset (HBD21)'

# Number of frames to extract from each video (you can set this to None if you want all frames)
frames_per_video = 30

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to extract frames from a video and save them as images
def extract_frames(video_path, output_folder, frames_to_extract=None):
    # Create a VideoCapture object
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Get total number of frames
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))  # Get the frame rate of the video

    # Calculate interval for extracting frames (if specified)
    if frames_to_extract:
        interval = max(1, frame_count // frames_to_extract)
    else:
        interval = 1  # Extract all frames

    # Loop to read frames
    count = 0
    extracted_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Save frame at the specified interval
        if count % interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{count:05d}.jpg")
            cv2.imwrite(frame_filename, frame)
            extracted_count += 1

        count += 1
        # Stop if we've extracted the specified number of frames
        if frames_to_extract and extracted_count >= frames_to_extract:
            break

    cap.release()

# Loop through Train and Test folders
for dataset_type in ['Train', 'Test']:
    dataset_dir = os.path.join(base_dir, dataset_type)

    # Loop through each class/category folder in the dataset (e.g., Abuse, Arrest, etc.)
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)

        # Check if the current item is a directory
        if not os.path.isdir(class_dir):
            continue

        # Loop through all video files in the current class folder
        for video_file in os.listdir(class_dir):
            video_path = os.path.join(class_dir, video_file)

            # Skip non-video files (if any)
            if not video_file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
                continue

            # Create a folder for each video under the correct class in the output directory
            output_class_dir = os.path.join(output_dir, dataset_type, class_name)
            if not os.path.exists(output_class_dir):
                os.makedirs(output_class_dir)

            # Create a folder specific to this video within the class folder
            video_name = os.path.splitext(video_file)[0]
            output_video_dir = os.path.join(output_class_dir, video_name)
            if not os.path.exists(output_video_dir):
                os.makedirs(output_video_dir)

            # Extract frames from the video and save them
            extract_frames(video_path, output_video_dir, frames_per_video)

print("Frames extraction with labels complete!")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import pickle
from tqdm import tqdm
import numpy as np
import random

In [ ]:
test_dir = '/content/drive/MyDrive/Human Behavior dataset (HBD21)/Train'
train_dir = '/content/drive/MyDrive/Human Behavior dataset (HBD21)/Test'

In [ ]:
# Define the categories and labels
categories_labels = {'assault_violence': 0, 'gun_violence': 1, 'normal_actions': 2, 'sabotage_violence': 3}

In [ ]:
import os
import cv2
from tqdm import tqdm

def load_data(base_dir, categories_labels):
    data = []

    # Go through each category
    for category, label in categories_labels.items():
        # The path to the category directory
        category_dir = os.path.join(base_dir, category)
        print(f"Checking category directory: {category_dir}")  # Debugging print

        # Make sure the directory exists
        if os.path.isdir(category_dir):
            # Go through each subfolder (video name) in the category directory
            for video_subfolder in os.listdir(category_dir):
                video_subfolder_path = os.path.join(category_dir, video_subfolder)
                print(f"  Checking video subfolder: {video_subfolder_path}")  # Debugging print

                # Ensure it's a directory (subfolder)
                if os.path.isdir(video_subfolder_path):
                    # Go through each image in the video subfolder
                    for filename in os.listdir(video_subfolder_path):
                        # Make sure the file is an image
                        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                            # The path to the image
                            image_path = os.path.join(video_subfolder_path, filename)
                            print(f"    Found image: {image_path}")  # Debugging print

                            try:
                                # Load the image
                                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

                                # Check if the image was loaded successfully
                                if image is None:
                                    print(f"Error: Could not read image {image_path}")
                                    continue

                                # Resize the image
                                image = cv2.resize(image, (128, 128))

                                # Reshape the image to a 4D array (ImageDataGenerator requires 4D array)
                                image = image.reshape((1,) + image.shape + (1,))

                                # Add the image and its label to the data
                                data.append([image, label])
                            except Exception as e:
                                print(f"Error processing image {image_path}: {e}")

        else:
            print(f"Category directory {category_dir} does not exist!")  # Debugging print

    return data

# Directories where the data is stored
train_dir = '/content/drive/MyDrive/Human Behavior dataset (HBD21)/Train'
test_dir = '/content/drive/MyDrive/Human Behavior dataset (HBD21)/Test'

# Category labels (You can replace this with your actual category labels)
categories_labels = {'assault_violence': 0, 'gun_violence': 1, 'normal_actions': 2, 'sabotage_violence': 3}
# Load the training and test data
training_data = load_data(train_dir, categories_labels)
test_data = load_data(test_dir, categories_labels)

# Combine the training and test data
total_data = training_data + test_data

print(f"Loaded {len(total_data)} images.")


In [ ]:
len(total_data)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_sample_images(data, categories_labels, num_samples=5):
    # Reverse the categories_labels dictionary to map labels back to category names
    labels_to_categories = {v: k for k, v in categories_labels.items()}

    plt.figure(figsize=(10, 10))

    # Randomly select a few images to display
    indices = np.random.choice(len(data), num_samples, replace=False)

    for i, idx in enumerate(indices):
        image, label = data[idx]

        # Reshape the image to display correctly (50x50 in your case)
        image = image.reshape(128, 128)

        # Get the corresponding category name for the label
        category_name = labels_to_categories[label]

        # Display the image
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(image, cmap='gray')
        plt.title(category_name)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Assuming you have a dictionary 'categories_labels'
# e.g., categories_labels = {'Abuse': 0, 'Arrest': 1, 'Arson': 2, ...}

# Display 5 sample images from total_data with their categorical labels
show_sample_images(total_data, categories_labels, num_samples=5)


In [ ]:
!sudo pip install keras
!pip install np_utils
!pip install pydot


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers import concatenate
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger, Callback
import time


# Initialize lists to store the images and the labels
images = []
labels = []

# Go through each image and its label in the total_data
for image, label in total_data:
    images.append(image)
    labels.append(label)

# Convert the lists into numpy arrays
images = np.array(images)
labels = np.array(labels)
print(images.shape)

# Reshape images if necessary for AlexNet
images_alex = images.reshape(images.shape[0], 128, 128, 1)  # Assuming grayscale images of size 128x128

# Set a seed for reproducibility
seed = 42

# Split the data into training and testing sets for CNN and AlexNet
train_images_cnn, test_images_cnn, train_labels_cnn, test_labels_cnn = train_test_split(images, labels, test_size=0.3, random_state=seed)
train_images_alex, test_images_alex, train_labels_alex, test_labels_alex = train_test_split(images_alex, labels, test_size=0.3, random_state=seed)

# Convert labels to categorical for CNN and AlexNet
train_labels = np_utils.to_categorical(train_labels_cnn, len(categories_labels))
test_labels = np_utils.to_categorical(test_labels_cnn, len(categories_labels))
train_labels_alex = np_utils.to_categorical(train_labels_alex, len(categories_labels))
test_labels_alex = np_utils.to_categorical(test_labels_alex, len(categories_labels))

# CNN Model
model_CNN = Sequential()
model_CNN.add(Conv2D(64, kernel_size=(3, 3), padding='same', input_shape=(128, 128, 1)))
model_CNN.add(LeakyReLU(alpha=0.1))
model_CNN.add(MaxPooling2D((2, 2), padding='same'))
model_CNN.add(Dropout(0.25))
model_CNN.add(Conv2D(128, (3, 3), padding='same'))
model_CNN.add(LeakyReLU(alpha=0.1))
model_CNN.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model_CNN.add(Dropout(0.25))
model_CNN.add(Conv2D(256, (3, 3), padding='same'))
model_CNN.add(LeakyReLU(alpha=0.1))
model_CNN.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model_CNN.add(Dropout(0.4))
model_CNN.add(Flatten())
model_CNN.add(Dense(256))
model_CNN.add(LeakyReLU(alpha=0.1))
model_CNN.add(Dropout(0.5))

# AlexNet Model
model_alexnet = Sequential()
model_alexnet.add(Conv2D(filters=96, input_shape=(128, 128, 1), kernel_size=(11, 11), strides=(4, 4), padding='same'))
model_alexnet.add(LeakyReLU(alpha=0.1))
model_alexnet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model_alexnet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
model_alexnet.add(LeakyReLU(alpha=0.1))
model_alexnet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model_alexnet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
model_alexnet.add(LeakyReLU(alpha=0.1))
model_alexnet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
model_alexnet.add(LeakyReLU(alpha=0.1))
model_alexnet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
model_alexnet.add(LeakyReLU(alpha=0.1))
model_alexnet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model_alexnet.add(Flatten())
model_alexnet.add(Dense(4096))
model_alexnet.add(LeakyReLU(alpha=0.1))
model_alexnet.add(Dropout(0.5))
model_alexnet.add(Dense(4096))
model_alexnet.add(LeakyReLU(alpha=0.1))
model_alexnet.add(Dropout(0.5))
model_alexnet.add(Dense(1000))
model_alexnet.add(LeakyReLU(alpha=0.1))
model_alexnet.add(Dropout(0.5))

# Combine CNN and AlexNet model
nb_classes = 4
combined = concatenate([model_CNN.output, model_alexnet.output], axis=-1)
output = Dense(nb_classes, activation='softmax')(combined)
model_final = Model(inputs=[model_CNN.input, model_alexnet.input], outputs=output)

# Plot and compile the model
plot_model(model_final, to_file='model_plot_alexnet.png', show_shapes=True, show_layer_names=True)

# Define Hybrid Optimizer Callback
class HybridOptimizer(Callback):
    def __init__(self, sgd, adam, switch_epoch):
        super(HybridOptimizer, self).__init__()
        self.sgd = sgd
        self.adam = adam
        self.switch_epoch = switch_epoch

    def on_epoch_begin(self, epoch, logs=None):
        if epoch < self.switch_epoch:
            print(f"Epoch {epoch + 1}: Using Adam optimizer")
            self.model.optimizer = self.adam
        else:
            print(f"Epoch {epoch + 1}: Switching to SGD optimizer")
            self.model.optimizer = self.sgd

# Initialize optimizers
adam = Adam(learning_rate=0.001)
sgd = SGD(learning_rate=0.01, momentum=0.9)

# Use Adam for the first 10 epochs, then switch to SGD
switch_epoch = 10

# Initialize hybrid optimizer callback
hybrid_optimizer_callback = HybridOptimizer(sgd, adam, switch_epoch)

# Callbacks
csv_logger = CSVLogger('training_alexnet.log', separator=',', append=False)
mc = ModelCheckpoint('Model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

# Compile model with Adam optimizer initially
model_final.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Training
time1 = time.time()
history = model_final.fit([train_images_cnn, train_images_alex], train_labels_alex,
                          batch_size=64, epochs=20,
                          validation_data=([test_images_cnn, test_images_alex], test_labels_alex),
                          callbacks=[mc, csv_logger, hybrid_optimizer_callback])
print(("Training time=", time.time() - time1))

# Save training history
np.save("Model_history.npy", history.history)


In [ ]:
from keras.models import load_model
fashion_model = load_model('/content/Model.h5') # load model
fashion_model.summary() # summarize model.

from contextlib import redirect_stdout
with open('/content/Model.h5'+".xls", 'w') as f:
    with redirect_stdout(f):
        fashion_model.summary()

val_loss, val_accuracy=fashion_model.evaluate([test_images_cnn, test_images_alex] ,test_labels) ## to get test accuracy and losses
print(val_loss, val_accuracy)

In [ ]:
time2=time.time()
predict_prob=fashion_model.predict([test_images_cnn, test_images_alex])
y_pred=np.argmax(predict_prob,axis=1)
print ('classification time:', time.time()-time2)

##print (y_pred)
y_true=np.argmax(test_labels, axis=1) # Use test_labels instead of test_labels_cnn
from sklearn.metrics import precision_recall_fscore_support as score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_true, y_pred)
print (cm)
print(classification_report(y_true, y_pred))

precision = precision_score(y_true, y_pred, average='weighted')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_true, y_pred, average='weighted')
print('Recall: %f' % recall)
# f1: tp / (tp + fp + fn)
f1 = f1_score(y_true, y_pred, average='weighted')
print('F1 score: %f' % f1)
#-----------  IoU
from sklearn.metrics import jaccard_score
print ('IoU:', jaccard_score(y_true, y_pred, average='micro'))


test_eval = fashion_model.evaluate([test_images_cnn, test_images_alex], test_labels) # Use test_labels instead of test_labels_cnn

loss, accuracy = fashion_model.evaluate([train_images_cnn, train_images_alex], train_labels) # Use train_labels instead of train_labels_cnn
print('loss_train: ', loss, 'accuracy_train: ', accuracy)
print('Test loss:', test_eval[0], 'Test accuracy:', test_eval[1])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_validation_images(test_images_cnn, test_images_lstm, y_true, y_pred, categories_labels, num_samples=5):
    # Reverse the categories_labels dictionary to map labels back to category names
    labels_to_categories = {v: k for k, v in categories_labels.items()}

    plt.figure(figsize=(15, 15))

    # Randomly select a few images to display
    indices = np.random.choice(len(test_images_cnn), num_samples, replace=False)

    for i, idx in enumerate(indices):
        image = test_images_cnn[idx]

        # Get the true and predicted labels
        true_label = labels_to_categories[y_true[idx]]
        predicted_label = labels_to_categories[y_pred[idx]]

        # Reshape the image to display correctly (50x50 in your case)
        image = image.reshape(128, 128)

        # Display the image
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(image, cmap='gray')
        plt.title(f"True: {true_label}\nPredicted: {predicted_label}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Assuming y_true and y_pred are already calculated
# y_true is already in the correct format, no need to apply argmax
# y_true = np.argmax(test_labels_cnn, axis=1)  # Convert one-hot encoded labels to class integers

# Display 5 validation images with their true and predicted labels
show_validation_images(test_images_cnn, test_images_alex, y_true, y_pred, categories_labels, num_samples=5)

In [ ]:
import matplotlib.pyplot as plt

history_dict=history.history
loss_values=history_dict['loss']
val_loss_values=history_dict['val_loss']
acc_values=history_dict['accuracy']
val_acc_values=history_dict['val_accuracy']
epochs=range(1, len(acc_values)+1)
def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points
loss_values=smooth_curve(loss_values)
val_loss_values=smooth_curve(val_loss_values)
acc_values=smooth_curve(acc_values)
val_acc_values=smooth_curve(val_acc_values)

font = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'normal',
        'size'   : 12}


plt.plot(epochs, acc_values, 'r-', label='Training acc')
plt.plot(epochs, val_acc_values, 'g', label='Validation acc')
plt.title('Training and Validation acc', fontdict=font)
plt.xlabel('Epochs', fontdict=font)
plt.ylabel('Accuracy', fontdict=font)
plt.legend()
plt.savefig("accuracy"+'CNN_AlexNet'+".png")
plt.show()

plt.plot(epochs, loss_values, 'b-', label='Training loss')
plt.plot(epochs, val_loss_values, 'r', label='Validation loss')
plt.title('Training and Validation loss', fontdict=font)
plt.xlabel('Epochs',fontdict=font)
plt.ylabel('Loss',fontdict=font)
plt.legend()
plt.savefig("loss"+'CNN_AlexNet'+".png")
plt.show()

In [ ]:
class_names = ['assault_violence', 'gun_violence', 'normal_actions', 'sabotage_violence']


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

font = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'normal',
        'size'   : 14}
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')
plt.savefig('confusion matrix1'+'CNN_AlexNet'+'.png')
plt.show()
# Plot normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
plt.savefig('confusion matrix2'+'CNN_AlexNet'+'.png')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Step 1: Make predictions using the trained model on test data
predictions = model_final.predict([test_images_cnn, test_images_alex])

# Step 2: Convert predictions from one-hot encoding back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Convert true labels (from one-hot encoding) back to class labels
true_classes = np.argmax(test_labels, axis=1)

# Step 3: Generate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Step 4: Visualize the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=categories_labels)
disp.plot(cmap=plt.cm.Blues)

# Step 5: Show the plot
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, precision_score, recall_score, f1_score, jaccard_score, roc_auc_score
from sklearn.preprocessing import label_binarize

# Timing the predictions
time2 = time.time()
predict_prob = fashion_model.predict([test_images_cnn, test_images_alex])
y_pred = np.argmax(predict_prob, axis=1)
print('Classification time:', time.time() - time2)

# True labels
y_true = np.argmax(test_labels, axis=1)

# Classification metrics
cm = confusion_matrix(y_true, y_pred)
print(cm)
print(classification_report(y_true, y_pred))

# Precision, recall, F1
precision = precision_score(y_true, y_pred, average='weighted')
print('Precision: %f' % precision)
recall = recall_score(y_true, y_pred, average='weighted')
print('Recall: %f' % recall)
f1 = f1_score(y_true, y_pred, average='weighted')
print('F1 score: %f' % f1)

# Intersection over Union (IoU)
print('IoU:', jaccard_score(y_true, y_pred, average='micro'))

# Model evaluation
test_eval = fashion_model.evaluate([test_images_cnn, test_images_alex], test_labels)
loss, accuracy = fashion_model.evaluate([train_images_cnn, train_images_alex], train_labels)
print('Train loss:', loss, 'Train accuracy:', accuracy)
print('Test loss:', test_eval[0], 'Test accuracy:', test_eval[1])

# --------- ROC-AUC Curve ---------

# Step 1: Binarize the true labels for the one-vs-rest approach
n_classes = len(np.unique(y_true))  # Total number of classes
y_true_bin = label_binarize(y_true, classes=range(n_classes))

# Step 2: Calculate ROC curve and AUC for each class
fpr = dict()  # False Positive Rate for each class
tpr = dict()  # True Positive Rate for each class
roc_auc = dict()  # AUC for each class

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], predict_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Step 3: Plot the ROC curve for each class with AUC
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

# Plot diagonal line representing random guessing
plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')

# Customize the plot
plt.title('ROC-AUC Curve for Suspecious Activity Detection')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True)

# Step 4: Show the plot
plt.show()

# Step 5: Calculate and print overall AUC score (micro and macro average)
micro_roc_auc = roc_auc_score(y_true_bin, predict_prob, average='micro')
macro_roc_auc = roc_auc_score(y_true_bin, predict_prob, average='macro')

print(f'Micro-average AUC: {micro_roc_auc:.2f}')
print(f'Macro-average AUC: {macro_roc_auc:.2f}')
